In [1]:
#MERGED DATA

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import utils
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error # for calculating the cost function
from sklearn.ensemble import RandomForestRegressor # for building the model
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score


data = pd.read_csv('/Users/selinzobu/Desktop/TILES/111OMFITBASES111.csv', parse_dates = ['Timestamp'])
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
#fitbit["Timestamp"] = pd.to_datetime(fitbit["Timestamp"], utc=True)
#print(data.columns)

bases = pd.read_csv('/Users/selinzobu/Desktop/TILES/1BASESTRESSD.csv', parse_dates = ['Timestamp'])
bases = bases.loc[:, ~bases.columns.str.contains('^Unnamed')]

FOM = pd.read_csv('/Users/selinzobu/Desktop/TILES/1FITOMSTRESSD.csv', parse_dates = ['Timestamp'])
FOM = FOM.loc[:, ~FOM.columns.str.contains('^Unnamed')]

X = data.iloc[:,2:65]  #independent columns
X = X.drop('days', axis = 1)
y = data.iloc[:,-1]    #target column 

X_base = bases.drop(['participant_id', 'Timestamp', 'mstressd'], axis = 1)  #independent columns
y_base = bases['mstressd']    #target column 

X_FOM = FOM.drop(['participant_id', 'Timestamp', 'mstressd', 'days'], axis = 1)  #independent columns
y_FOM = FOM['mstressd']   #target column 

#for col in X.columns:
 #   X[col] = X[col].apply(pd.to_numeric, errors='coerce')

In [2]:
print(len(data))
print(type(data))
print(len(data.groupby(['Timestamp', 'participant_id'])['mstressd'].unique()))

2989
<class 'pandas.core.frame.DataFrame'>
2984


In [3]:
X2 = X[['AvgHeartRateS', 'RMSStdDev_msM', 'AvgBreathingRateM', 'AvgZAccel_gM', 'StdDevBreathingRateM', 'Fat Burn_caloriesOut', 'NumberSteps', 'iod_ID', 'iod_OD', 'ocb', 'pan_NegAffect', 'stai', 'rand_EmotionalWellbeing', 'swls', 'pss', 'mpfi_Flexibility_Defusion', 'pcq_Hope', 'pcq_Optimism', 'chss_ChallengeStressors', 'chss_HindranceStressors']]

X2_FOM = X_FOM[['Cardio_caloriesOut', 'AvgHeartRateS', 'RMSStdDev_msM', 'AvgBreathingRateM', 'AvgYAccel_gM', 'AvgZAccel_gM', 'StdDevBreathingRateM', 'StdDevBreathingRateS', 'StdDevGForceM', 'StdDevYAccel_gM']]

X2_base = X_base[['rand_EmotionalWellbeing', 'ocb', 'pcq_Hope', 'mpfi_Flexibility_Defusion', 'mpfi_Flexibility_SelfAsContext', 'iod_ID', 'mpfi_Flexibility_PresentMomentAwareness', 'swls', 'chss_ChallengeStressors', 'stai', 'pan_NegAffect']]

In [4]:
print(X2.shape, X2_FOM.shape, X2_base.shape, y.shape)

(2989, 20) (3126, 10) (7868, 11) (2989,)


In [5]:
X2_train, X2_test, y_train, y_test = train_test_split(
    X2, y, test_size=0.3, random_state=42)

X2b_train, X2b_test, yb_train, yb_test = train_test_split(
    X2_base, y_base, test_size=0.3, random_state=42)

X2FOM_train, X2FOM_test, yFOM_train, yFOM_test = train_test_split(
    X2_FOM, y_FOM, test_size=0.3, random_state=42)

In [6]:
print('X2_train:', X2_train.shape, 'X2_test:', X2_test.shape, 'y_train:', y_train.shape,'y_test:', y_test.shape)
#print(y_test.head())
print('X2b_train:', X2b_train.shape, 'X2b_test:', X2b_test.shape, 'yb_train:', yb_train.shape,'yb_test:', yb_test.shape)
#print(yb_test.head())
print('X2FOM_train:', X2FOM_train.shape, 'X2FOM_test:', X2FOM_test.shape, 'yFOM_train:', yFOM_train.shape,'yFOM_test:', yFOM_test.shape)
#print(yFOM_test.head())

X2_train: (2092, 20) X2_test: (897, 20) y_train: (2092,) y_test: (897,)
X2b_train: (5507, 11) X2b_test: (2361, 11) yb_train: (5507,) yb_test: (2361,)
X2FOM_train: (2188, 10) X2FOM_test: (938, 10) yFOM_train: (2188,) yFOM_test: (938,)


In [7]:
print(y_train.mean(), y_train.shape, yb_train.mean(),yb_train.shape, yFOM_train.mean(), yFOM_train.shape)

1.902963671128107 (2092,) 1.803068821499909 (5507,) 1.9079067641681902 (2188,)


In [11]:
#The baseline prediction for our case can be the historical stress score averages. 
#In other words, our baseline is the error we would get if we simply predicted the average
#stress score for all days.

# The baseline predictions are the historical averages
Mbaseline_preds = y_train.mean()
Bbaseline_preds = yb_train.mean()
FOMbaseline_preds = yFOM_train.mean()

# Baseline errors, and display average baseline error
Mbaseline_errors = abs(Mbaseline_preds - y_test)
Bbaseline_errors = abs(Bbaseline_preds - yb_test)
FOMbaseline_errors = abs(FOMbaseline_preds - yFOM_test)


print('TAverage baseline error merged: ', round(np.mean(Mbaseline_errors), 2))
print('TAverage baseline error baseline: ', round(np.mean(Bbaseline_errors), 2))
print('TAverage baseline error FOM: ', round(np.mean(FOMbaseline_errors), 2))


print(len(Mbaseline_errors), len(Bbaseline_errors), len(FOMbaseline_errors))

Mbaseline_errors = list(Mbaseline_errors)
Bbaseline_errors = list(Bbaseline_errors)
FOMbaseline_errors = list(FOMbaseline_errors)


TAverage baseline error merged:  0.75
TAverage baseline error baseline:  0.75
TAverage baseline error FOM:  0.73
897 2361 938


In [9]:
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
  
# create regressor object
regressor = RandomForestRegressor(n_estimators = 200, random_state = 0)
  
# fit the regressor with x and y data
regressor.fit(X2_train, y_train)  

#model.fit(Xb_train, yb_train)
#model.fit(XFOM_train, yFOM_train)

RandomForestRegressor(n_estimators=200, random_state=0)

In [14]:
# BASELINE
#Fitting Random Forest Regression to the dataset
# import the regressor

# create regressor object
regressor = RandomForestRegressor(n_estimators = 200, random_state = 0)
  
# fit the regressor with x and y data
regressor.fit(X2b_train, yb_train)  


RandomForestRegressor(n_estimators=200, random_state=0)

In [18]:
# FOM
#Fitting Random Forest Regression to the dataset
# import the regressor

# create regressor object
regressor = RandomForestRegressor(n_estimators = 200, random_state = 0)
  
# fit the regressor with x and y data
regressor.fit(X2FOM_train, yFOM_train)  

RandomForestRegressor(n_estimators=200, random_state=0)

In [13]:
# Predicting the target values of the test set
y2_pred = regressor.predict(X2_test)

# RMSE (Root Mean Square Error)
rmse = float(format(np.sqrt(mean_squared_error(y_test, y2_pred)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  0.771


In [15]:
# BASELINE
#Predicting the target values of the test set
y2b_pred = regressor.predict(X2b_test)

# RMSE (Root Mean Square Error)
rmse = float(format(np.sqrt(mean_squared_error(yb_test, y2b_pred)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  0.762


In [19]:
# FOM
#Predicting the target values of the test set
y2FOM_pred = regressor.predict(X2FOM_test)

# RMSE (Root Mean Square Error)
rmse = float(format(np.sqrt(mean_squared_error(yFOM_test, y2FOM_pred)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  0.835


In [32]:
# Calculate the absolute errors
errors = abs(y2_pred - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.6 degrees.


In [140]:
# BASELINE
#Calculate the absolute errors
errors = abs(y2b_pred - yb_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.59 degrees.


In [20]:
# FOM
#Calculate the absolute errors
errors = abs(y2FOM_pred - yFOM_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.68 degrees.


In [33]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 64.14 %.


In [141]:
#BASELINE
#Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / yb_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 62.39 %.


In [154]:
#FOM
#Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / yFOM_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 56.52 %.


In [143]:
feature_list = list(X2.columns)
print(feature_list)

['AvgHeartRateS', 'RMSStdDev_msM', 'AvgBreathingRateM', 'AvgZAccel_gM', 'StdDevBreathingRateM', 'Fat Burn_caloriesOut', 'NumberSteps', 'iod_ID', 'iod_OD', 'ocb', 'pan_NegAffect', 'stai', 'rand_EmotionalWellbeing', 'swls', 'pss', 'mpfi_Flexibility_Defusion', 'pcq_Hope', 'pcq_Optimism', 'chss_ChallengeStressors', 'chss_HindranceStressors']


In [72]:
#!pip install pydot
#!pip install graphviz
#!install gprof2dot

In [41]:
# Get numerical feature importances
importances = list(regressor.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: StdDevBreathingRateM Importance: 0.1
Variable: RMSStdDev_msM        Importance: 0.09
Variable: AvgBreathingRateM    Importance: 0.09
Variable: AvgZAccel_gM         Importance: 0.09
Variable: NumberSteps          Importance: 0.09
Variable: AvgHeartRateS        Importance: 0.08
Variable: Fat Burn_caloriesOut Importance: 0.08
Variable: chss_ChallengeStressors Importance: 0.07
Variable: mpfi_Flexibility_Defusion Importance: 0.06
Variable: rand_EmotionalWellbeing Importance: 0.04
Variable: iod_ID               Importance: 0.03
Variable: pan_NegAffect        Importance: 0.03
Variable: stai                 Importance: 0.03
Variable: iod_OD               Importance: 0.02
Variable: ocb                  Importance: 0.02
Variable: swls                 Importance: 0.02
Variable: pss                  Importance: 0.02
Variable: pcq_Hope             Importance: 0.02
Variable: pcq_Optimism         Importance: 0.02
Variable: chss_HindranceStressors Importance: 0.02


In [144]:
# BASELINE
#Get numerical feature importances
importances = list(regressor.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: iod_OD               Importance: 0.17
Variable: AvgHeartRateS        Importance: 0.11
Variable: Fat Burn_caloriesOut Importance: 0.11
Variable: pan_NegAffect        Importance: 0.11
Variable: iod_ID               Importance: 0.1
Variable: RMSStdDev_msM        Importance: 0.08
Variable: ocb                  Importance: 0.08
Variable: AvgZAccel_gM         Importance: 0.07
Variable: AvgBreathingRateM    Importance: 0.06
Variable: StdDevBreathingRateM Importance: 0.06
Variable: NumberSteps          Importance: 0.05


[None, None, None, None, None, None, None, None, None, None, None]

In [155]:
# FOM
#Get numerical feature importances
importances = list(regressor.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: AvgBreathingRateM    Importance: 0.12
Variable: AvgZAccel_gM         Importance: 0.11
Variable: StdDevBreathingRateM Importance: 0.11
Variable: Fat Burn_caloriesOut Importance: 0.11
Variable: RMSStdDev_msM        Importance: 0.1
Variable: NumberSteps          Importance: 0.1
Variable: iod_ID               Importance: 0.1
Variable: iod_OD               Importance: 0.1
Variable: ocb                  Importance: 0.1
Variable: AvgHeartRateS        Importance: 0.06


[None, None, None, None, None, None, None, None, None, None]

In [16]:
#Training results and best parameters

def model_train(X_t, y_t):
# Perform Grid-Search
    param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 400]}

    grid_search = GridSearchCV(
        estimator=RandomForestRegressor(random_state =0),
        param_grid=param_grid,
        cv=KFold(n_splits=5, shuffle=True, random_state=1),
        scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
    grid_result = grid_search.fit(X_t, y_t)
    best_params = grid_result.best_params_

    
    rf = RandomForestRegressor(bootstrap = grid_search.best_params_['bootstrap'],
                           n_estimators = grid_search.best_params_['n_estimators'],
                           max_features = grid_search.best_params_['max_features'],
                           max_depth = grid_search.best_params_['max_depth'],
                           min_samples_leaf = grid_search.best_params_['min_samples_leaf'],
                           min_samples_split = grid_search.best_params_['min_samples_split'],
                           random_state = 0)
# Perform K-Fold CV
    trainscores = cross_val_score(rf, X_t, y_t, cv=10, scoring='neg_mean_absolute_error')
    train_n_scores = cross_val_score(rf, X_t, y_t,
                           cv=KFold(n_splits=5, shuffle=True, random_state=1)).mean()
    
    return 'bestparameters;', best_params, 'MAE:', trainscores.mean(), train_n_scores

In [14]:
#MULTIMODAL TRAIN RESULTS

print(model_train(X2_train, y_train))

({'bootstrap': True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 400}, -0.5762064386433006, 0.29703392472767015)


In [15]:
#BASELINE TRAIN RESULTS

print(model_train(X2b_train, yb_train))

({'bootstrap': True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 5, 'min_samples_split': 12, 'n_estimators': 100}, -0.574937723095504, 0.28458706604493317)


In [17]:
#FOM TRAIN RESULTS

print(model_train(X2FOM_train, yFOM_train))

('bestparameters;', {'bootstrap': True, 'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 100}, 'MAE:', -0.6942228831994853, 0.12596278346944778)


In [12]:
#Multimodal Errors

MRF_errors = []
    

best_param = {'bootstrap': True, 'max_depth': 80, 'max_features': 3, 
                'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 400}
    
rf = RandomForestRegressor(bootstrap = best_param['bootstrap'],
                           n_estimators = best_param['n_estimators'],
                           max_features = best_param['max_features'],
                           max_depth = best_param['max_depth'],
                           min_samples_leaf = best_param['min_samples_leaf'],
                           min_samples_split = best_param['min_samples_split'],
                           random_state = 0)

rf.fit(X2_train, y_train.ravel())
y_pred = rf.predict(X2_test)

MRF_errors = abs(y_pred - y_test)


mae = mean_absolute_error(y_test, y_pred)

print("\nMAE:", mae, len(MRF_errors))


MAE: 0.589500167854654 897


In [13]:
def mergedmodel_test(X_t, y_t, X_test, y_test):
    best_param = {'bootstrap': True, 'max_depth': 80, 
                  'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 10,
                  'n_estimators': 400}
    
    rf = RandomForestRegressor(bootstrap = best_param['bootstrap'],
                           n_estimators = best_param['n_estimators'],
                           max_features = best_param['max_features'],
                           max_depth = best_param['max_depth'],
                           min_samples_leaf = best_param['min_samples_leaf'],
                           min_samples_split = best_param['min_samples_split'],
                           random_state = 0)
# Perform K-Fold CV
    scores = cross_val_score(rf, X_test, y_test, cv=10, scoring='neg_mean_absolute_error')
    n_scores = cross_val_score(rf, X_test, y_test,
                           cv=KFold(n_splits=5, shuffle=True, random_state=1)).mean()

    rf.fit(X_t, y_t)
    y_pred = rf.predict(X_test)
    rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)), '.3f'))
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    

    return scores.mean(), "\RMSE: ", rmse, '\nMAE: ', mae, '\nMSE: ', mse, n_scores

In [15]:
#MULTIMODAL -TEST RESULTS 
print(mergedmodel_test(X2_train, y_train, X2_test, y_test))

(-0.6277521718453076, '\\RMSE: ', 0.748, '\nMAE: ', 0.589500167854654, '\nMSE: ', 0.5591977088624736, 0.23996831875476543)


In [20]:
#Baseline Errors

BRF_errors = []
    
best_param = {'bootstrap': True, 'max_depth': 80, 'max_features': 3, 
                   'min_samples_leaf': 5, 'min_samples_split': 12, 'n_estimators': 100}
    
rf = RandomForestRegressor(bootstrap = best_param['bootstrap'],
                           n_estimators = best_param['n_estimators'],
                           max_features = best_param['max_features'],
                           max_depth = best_param['max_depth'],
                           min_samples_leaf = best_param['min_samples_leaf'],
                           min_samples_split = best_param['min_samples_split'],
                           random_state = 0)

rf.fit(X2b_train, yb_train.ravel())
y_pred = rf.predict(X2b_test)

BRF_errors = abs(y_pred - yb_test)


mae = mean_absolute_error(yb_test, y_pred)

print("\nMAE:", mae, len(BRF_errors))


MAE: 0.5881430306797508 2361


In [16]:
def basemodel_test(X_t, y_t, X_test, y_test):
    best_param = {'bootstrap': True, 'max_depth': 80, 'max_features': 3, 
                   'min_samples_leaf': 5, 'min_samples_split': 12, 'n_estimators': 100}
    
    rf = RandomForestRegressor(bootstrap = best_param['bootstrap'],
                           n_estimators = best_param['n_estimators'],
                           max_features = best_param['max_features'],
                           max_depth = best_param['max_depth'],
                           min_samples_leaf = best_param['min_samples_leaf'],
                           min_samples_split = best_param['min_samples_split'],
                           random_state = 0)
# Perform K-Fold CV
    scores = cross_val_score(rf, X_test, y_test, cv=10, scoring='neg_mean_absolute_error')
    n_scores = cross_val_score(rf, X_test, y_test,
                           cv=KFold(n_splits=5, shuffle=True, random_state=1)).mean()

    rf.fit(X_t, y_t)
    y_pred = rf.predict(X_test)
    rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)), '.3f'))
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    

    return scores.mean(), "\RMSE: ", rmse, '\nMAE: ', mae, '\nMSE: ', mse, n_scores

In [17]:
#BASELINE - TEST RESULTS

print(basemodel_test(X2b_train, yb_train, X2b_test, yb_test))

(-0.5990334144625866, '\\RMSE: ', 0.761, '\nMAE: ', 0.5881430306797508, '\nMSE: ', 0.5789009689613209, 0.24408892416043626)


In [21]:
#FOM Errors

FOMRF_errors = []
    
best_param = {'bootstrap': True, 'max_depth': 80, 'max_features': 2, 
                  'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 100}
    
rf = RandomForestRegressor(bootstrap = best_param['bootstrap'],
                           n_estimators = best_param['n_estimators'],
                           max_features = best_param['max_features'],
                           max_depth = best_param['max_depth'],
                           min_samples_leaf = best_param['min_samples_leaf'],
                           min_samples_split = best_param['min_samples_split'],
                           random_state = 0)

rf.fit(X2FOM_train, yFOM_train.ravel())
y_pred = rf.predict(X2FOM_test)

FOMRF_errors = abs(y_pred - yFOM_test)


mae = mean_absolute_error(yFOM_test, y_pred)

print("\nMAE:", mae, len(FOMRF_errors))


MAE: 0.6752463569759651 938


In [18]:
def FOMmodel_test(X_t, y_t, X_test, y_test):
    best_param = {'bootstrap': True, 'max_depth': 80, 'max_features': 2, 
                  'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 100}
    
    rf = RandomForestRegressor(bootstrap = best_param['bootstrap'],
                           n_estimators = best_param['n_estimators'],
                           max_features = best_param['max_features'],
                           max_depth = best_param['max_depth'],
                           min_samples_leaf = best_param['min_samples_leaf'],
                           min_samples_split = best_param['min_samples_split'],
                           random_state = 0)
# Perform K-Fold CV
    scores = cross_val_score(rf, X_test, y_test, cv=10, scoring='neg_mean_absolute_error')
    n_scores = cross_val_score(rf, X_test, y_test,
                           cv=KFold(n_splits=5, shuffle=True, random_state=1)).mean()

    rf.fit(X_t, y_t)
    y_pred = rf.predict(X_test)
    rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)), '.3f'))
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    

    return scores.mean(), "\RMSE: ", rmse, '\nMAE: ', mae, '\nMSE: ', mse, n_scores

In [19]:
#FOM - TEST RESULTS

print(FOMmodel_test(X2FOM_train, yFOM_train, X2FOM_test, yFOM_test))

(-0.6998335585495488, '\\RMSE: ', 0.826, '\nMAE: ', 0.6752463569759651, '\nMSE: ', 0.6825073887358436, 0.07745614694963285)


In [98]:
#Another attempt with CV and Grid Search

from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error


def rfr_model(X_t, y_t, X_te, y_te):
# Perform Grid-Search
    param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 400]}

    grid_search = GridSearchCV(
        estimator=RandomForestRegressor(random_state =0),
        param_grid=param_grid,
        cv=KFold(n_splits=5, shuffle=True, random_state=1),
        scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
    grid_result = grid_search.fit(X_t, y_t)
    best_params = grid_result.best_params_
    
   # rfr = RandomForestRegressor(max_depth=best_params["max_depth"], 
  #  n_estimators=best_params["n_estimators"],random_state=False, verbose=False)
    rf = RandomForestRegressor(bootstrap = grid_search.best_params_['bootstrap'],
                           n_estimators = grid_search.best_params_['n_estimators'],
                           max_features = grid_search.best_params_['max_features'],
                           max_depth = grid_search.best_params_['max_depth'],
                           min_samples_leaf = grid_search.best_params_['min_samples_leaf'],
                           min_samples_split = grid_search.best_params_['min_samples_split'],
                           random_state = 0)
# Perform K-Fold CV
    scores = cross_val_score(rf, X_t, y_t, cv=10, scoring='neg_mean_absolute_error')
    #n_scores = cross_val_score(rf, X_t, y_t,
                           cv=KFold(n_splits=5, shuffle=True, random_state=1)).mean()
    #rfr.fit(X_t, y_t)
    rf.fit(X_t, y_t)
    y_pred = rf.predict(X_te)
    rmse = float(format(np.sqrt(mean_squared_error(y_te, y_pred)), '.3f'))
    mae = mean_absolute_error(y_te, y_pred)
    mse = mean_squared_error(y_te, y_pred)
    

    return scores, "\RMSE: ", rmse, '\nMAE: ', mae, '\nMSE: ', mse

In [99]:
rfr_model(X2_train, y_train, X2_test, y_test)

(array([-0.59621487, -0.58066229, -0.59790671, -0.60217617, -0.58514583,
        -0.51574472, -0.57896342, -0.56638479, -0.58000919, -0.55885639]),
 '\\RMSE: ',
 0.748,
 '\nMAE: ',
 0.589500167854654,
 '\nMSE: ',
 0.5591977088624736,
 0.29703392472767015)

In [ ]:
#Boruta is based on random forest, which is an ensemble of decision trees. Decision trees
#do not need to be scaled, because selecting a split on one scale is equivalent to 
#selecting a re-scaled split on an alternative scale.

In [ ]:
# Visualising the Random Forest Regression results
  
# arrange for creating a range of values
# from min value of x to max 
# value of x with a difference of 0.01 
# between two consecutive values
X_grid = np.arrange(min(x), max(x), 0.01) 
  
# reshape for reshaping the data into a len(X_grid)*1 array, 
# i.e. to make a column out of the X_grid value                  
X_grid = X_grid.reshape((len(X_grid), 1))
  
# Scatter plot for original data
plt.scatter(x, y, color = 'blue')  
  
# plot predicted data
plt.plot(X_grid, regressor.predict(X_grid), 
         color = 'green') 
plt.title('Random Forest Regression')
plt.xlabel('Position level')
plt.ylabel('Salary')
plt.show()

In [31]:
# SIGNIFICANCE OF DIFFERENCE

import scipy.stats as stats
import os


print(len(FOMRF_errors), len(FOMbaseline_errors))
print(len(BRF_errors), len(Bbaseline_errors))
print(len(MRF_errors), len(Mbaseline_errors))


os.makedirs('/Users/selinzobu/Desktop/TILES/', exist_ok=True)  
MRF_errors.to_csv('/Users/selinzobu/Desktop/TILES/1MRFERRORS.csv') 
BRF_errors.to_csv('/Users/selinzobu/Desktop/TILES/1BRFERRORS.csv') 
FOMRF_errors.to_csv('/Users/selinzobu/Desktop/TILES/1FOMRFERRORS.csv') 

938 938
2361 2361
897 897


In [36]:
print(MRF_errors.shape)
print(MRF_errors)

(897,)
1011    0.198245
1913    0.170930
783     0.518097
1600    0.262431
324     0.260553
          ...   
1263    0.244976
2236    0.582756
2939    0.084449
632     1.115347
1763    0.606265
Name: mstressd, Length: 897, dtype: float64


In [23]:
#conduct the Wilcoxon-Signed Rank Test

Mdif = stats.wilcoxon(MRF_errors, Mbaseline_errors)
FOMdif = stats.wilcoxon(FOMRF_errors, FOMbaseline_errors)
Bdif = stats.wilcoxon(BRF_errors, Bbaseline_errors)

print(Mdif)
print(FOMdif)
print(Bdif)

WilcoxonResult(statistic=121107.0, pvalue=4.566381071990293e-25)
WilcoxonResult(statistic=169749.0, pvalue=1.2158871042288187e-09)
WilcoxonResult(statistic=839708.0, pvalue=7.023825678919594e-63)


In [26]:
"{:.28f}".format(float("7.023825678919594e-63"))

'0.0000000000000000000000000000'